In [1]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import os
from time import sleep
from time import time
import csv

# to display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

def api_pull():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
        'start':'1',
        'limit':'21',
        'convert':'USD'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': 'f3f0734d-8d9c-4a2e-be9e-82e0501f442e',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:  
        print(e) 


    final_df = pd.json_normalize(data['data'])
    final_df['timestamp'] = pd.to_datetime('now', utc=True)
    # 'utc=True' to avoid deprecation error
    return final_df

# checks for file existence and creates if not exist, and closes with 'pass'
if not os.path.isfile(r'C:\Users\zee-sama\OneDrive\Desktop\Data\Jupyter\optimizedCryptoAPI.csv'):
    with open(r'C:\Users\zee-sama\OneDrive\Desktop\Data\Jupyter\optimizedCryptoAPI.csv', 'w') as opt:
        pass
    
# automates call, creates an empty list, each 'call' result is appended to the result
data_frames = []
#for pulls in range(7):
    #data_frames.append(api_pull())
    #print("API pull completed successfully!")
    #sleep(86400) # delay

# decided to remove the loop and set it up in task scheduler, the trigger in task scheduler manages the loop
data_frames.append(api_pull())
print('Data collected successfully')

# print(data_frames)
 
final_d = pd.concat(data_frames)

# compile all returned API calls and writes with 'mode=w' to csv file if file exists, else if it's a newly created file, 'mode=a' to append data every call
# final_d.to_csv(r'C:\Users\zee-sama\OneDrive\Desktop\Data\Jupyter\optimizedCryptoAPI.csv', mode='w', header=True, index=False)
final_d.to_csv(r'C:\Users\zee-sama\OneDrive\Desktop\Data\Jupyter\optimizedCryptoAPI.csv', mode='a', header=True, index=False)
# print(final_d)
# reads file

Data collected successfully


In [3]:
import pandas as pd
check_frame = pd.read_csv(r'C:\Users\zee-sama\OneDrive\Desktop\Data\Jupyter\optimizedCryptoAPI.csv')
print(check_frame)
# '.groupby.(metric of interest, e.g mean)' similar to groupings in SQL
final_d1 = final_d.groupby('name', sort=True)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
#final_d1

      id             name symbol                  slug  num_market_pairs  \
0      1          Bitcoin    BTC               bitcoin             10139   
1   1027         Ethereum    ETH              ethereum              6690   
2    825           Tether   USDT                tether             50651   
3   1839              BNB    BNB                   bnb              1241   
4   3408         USD Coin   USDC              usd-coin             11332   
5     52              XRP    XRP                   xrp               925   
6     74         Dogecoin   DOGE              dogecoin               676   
7   2010          Cardano    ADA               cardano               760   
8   3890          Polygon  MATIC               polygon               643   
9   5426           Solana    SOL                solana               454   
10  6636         Polkadot    DOT          polkadot-new               494   
11  4687      Binance USD   BUSD           binance-usd              6517   
12  5994    

In [ ]:
# converts to series
final_d2 = final_d1.stack()
#final_d2

In [ ]:
# flips back dataframe (column title gets converted into rows)
final_d3 = final_d2.to_frame()
final_d3.count()

In [ ]:
# 
index = pd.Index(range(126))
final_d4 = final_d3.reset_index()
final_d4

In [ ]:
# rename column 2
final_d5 = final_d4.rename(columns={'level_1':'percent_change', 0:'values'})
final_d5

In [ ]:
final_d5['percent_change'] = final_d5['percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 
                                    'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d'],
                                   ['1h','24h','7d','30d','60d','90d'])
final_d5

In [ ]:
import seaborn as sns
import matplotlib as plt

sns.catplot(x = 'percent_change', y = 'values', hue='name',data=final_d5, kind='point')